# Network Modeling with SEIRS+ Notebook 

In [1]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import poisson
from seirsplus.models import *
from network_utils import *
from stats_utils import *
from intervention_utils import *

### Parameters

Here, we define the parameters taken from the Tucker model. These can be customized further and then the rest of the cells can be run. For now we are only using the population in isoboxes to run our experiments and create the network, but we will soon include the tent population as well!

In [2]:
n_pop = 18700

# Isoboxes
n_isoboxes = 812
pop_isoboxes = 8100
pop_per_isobox = 10
max_pop_per_isobox  = poisson.rvs(mu=pop_per_isobox, size=n_isoboxes) # According to the model, this is drawn through a poison distribution

# Tents
n_tents = 2650
pop_tents = 10600
pop_per_tent = 4

# Others 
n_bathrooms = 144
n_ethnic_groups = 8

# Isobox grid parameters - can be extended to tents as well
grid_width = 29
grid_height = 28

# We define neighboring isoboxes as any isobox within a range of 2 in the isobox grid
iso_proximity = 2

# Sample the population age, and parameter rates
sample_pop = sample_population(pop_isoboxes)

#### 1.2) Create tent graphs

In [ ]:
tent_weight = 0.98 # Edge weight for connections within each isobox
graph, nodes_per_isobox = create_graph(n_isoboxes, pop_isoboxes, max_pop_per_isobox, 
                                       edge_weight=iso_weight, label="household", age_list=list(sample_pop["age"]),
                                      n_ethnicities=n_ethnic_groups)

### Basic Network

#### 1) Create isobox graph - can be used to create tent graphs as well

In [ ]:
iso_weight = 0.98 # Edge weight for connections within each isobox
graph, nodes_per_isobox = create_graph(n_isoboxes, pop_isoboxes, max_pop_per_isobox, 
                                       edge_weight=iso_weight, label="household", age_list=list(sample_pop["age"]),
                                      n_ethnicities=n_ethnic_groups)

#### 2) Create the grid that will help with positioning when measuring proximity

In [ ]:
isobox_grid = create_grid(grid_width, grid_height)

#### 3) Connect the nodes that are within a certain degree of proximity - can be used for tents as well

In [ ]:
neighbor_weight = 0.017 # Edge weight for connections between isoboxes
graph = connect_neighbors(graph, n_isoboxes, nodes_per_isobox, isobox_grid, iso_proximity, neighbor_weight, "friendship")

#### 4) Connect the nodes that go to the food line - can be extended to all nodes
Assumption: 2 people from each isobox are randomly selected to get food, and then we connect each person from the food queue with the previous + next 5 people near them.

In [ ]:
food_weight = 0.407 # Edge weight for connections in the food queue 
graph = connect_food_queue(graph, nodes_per_isobox, food_weight, "food_bois") 

#### 5) Plot the basic network degrees

In [ ]:
min_G, max_G = min_degree(graph), max_degree(graph)
min_G, max_G

In [ ]:
plot_degree_distn(graph, max_degree=max_G)

### Quarantine/Social Distancing Network

#### 1) Create simple quarantine graph
NOTE: We should tweak the reduction scale depending on what we want our mean degree to be + the min_neighbors parameter depending on how many people we think a person will be in contact with during a social distancing scenario

In [ ]:
reduction_scale = 10 # Exponential distribution parameter. Setting it to 20 gives a mean of ~10, when the past mean was ~40
min_neighbors = 4 # The minimum number of connections that a person would hold in a distancing/quarantine scenario

In [ ]:
Q = custom_exponential_graph(graph, scale=reduction_scale, min_num_edges=min_neighbors)

#### 2) Plot quarantine graph degrees

In [ ]:
min_Q, max_Q = min_degree(Q), max_degree(Q)
min_Q, max_Q

In [ ]:
plot_degree_distn(Q, max_degree=max_Q)

### Interventions

With the interventions module, we can create an intervention with just a time step and a custom network referring to that intervention, as well as remove/edit them from the list. The method get_checkpoints() will allow us to get the dictionary to be fed to the SEIRS+ model

In [ ]:
interventions = Interventions()

In [ ]:
interventions.add(15, Q)
interventions.add(45, graph)
interventions.add("dummy", Q)

In [ ]:
interventions.get_checkpoints()

In [ ]:
interventions.remove('dummy')

In [ ]:
interventions.get_checkpoints()

In [ ]:
transmission_rate = 1.28
progression_rate = 1/5.1
recovery_rate = 0.056 # Approx 1/18 -> Recovery occurs after 18 days
hosp_rate = 1/11.4 #1/6.3 # From Tucker Model
crit_rate = 0.3 # From camp_params

prob_global_contact = 1
prob_detected_global_contact = 1



prob_hosp_to_critical = list(sample_pop["death_rate"]/sample_pop["prob_hospitalisation"])
prob_asymptomatic = list(1 - sample_pop["prob_symptomatic"])
prob_symp_to_hosp = list(sample_pop["prob_hospitalisation"])

init_symp_cases = 1
init_asymp_cases = 0

In [ ]:
# checkpoints = interventions.get_checkpoints()

ref_model = SymptomaticSEIRSNetworkModel(G=graph, beta=transmission_rate, sigma=progression_rate, gamma=recovery_rate, 
                                         lamda=progression_rate, mu_H=crit_rate, eta=hosp_rate, p=prob_global_contact, a=prob_asymptomatic, f=prob_hosp_to_critical, 
                                         h=prob_symp_to_hosp, q=prob_detected_global_contact, initI_S=init_symp_cases, initI_A=init_asymp_cases, store_Xseries=True)

fig_name = f"SymptomaticModel_IsoWeight={iso_weight}_NeighWeight={neighbor_weight}_FoodWeight={food_weight}_ + \
TransRate={transmission_rate}_RecRate={recovery_rate}_ProgRate={progression_rate}_HospRate={hosp_rate}_initI_S={init_symp_cases}_initI_A={init_asymp_cases}"

ref_model.run(T=100, verbose=True)#, checkpoints=checkpoints)

In [ ]:
ref_model.run(T=150, verbose=True)

In [ ]:
fig, ax = ref_model.figure_basic()#vlines=interventions.get_checkpoints()['t'])
fig.savefig(f"plots/{fig_name}_figBasic.png")

In [ ]:
fig, ax = ref_model.figure_infections()#vlines=interventions.get_checkpoints()['t'])
fig.savefig(f"plots/{fig_name}_figInfections.png")

### Extracting the actual X and Y lists from the plots

In [ ]:
# len(np.ma.masked_where(ref_model.numF<=0, ref_model.tseries))
series = ref_model.numI_S
tseries = ref_model.tseries
topstack = np.zeros_like(tseries)


masked_array = np.ma.masked_where(series<=0, tseries)
masked_array


In [ ]:
# Possibility: we convert all the time series value "t" to int and that's how we have days? 
len(set([int(masked_array.data[i]) for i in range(len(masked_array)) if not masked_array.mask[i]]))


In [ ]:
ax.lines[3].get_ydata().data[::int(ref_model.numNodes/100)]

In [ ]:
len(ref_model.tseries)